# Re-Ranking

In the context of RAG (Retrieval-Augmented Generation), reranking of retrieval results is a crucial step that refines the initial set of retrieved documents based on their relevance to the input query. This process involves re-scoring the retrieved documents using a more sophisticated model, such as a cross-encoder, to better capture the semantic similarity between the query and the documents. The reranked list of documents is then used as input for the generation model, ensuring that the most relevant and accurate information is utilized to generate the final output.

![Cross Encoder Image](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/CrossEncoder.png)


Read more [here](https://www.sbert.net/examples/applications/retrieve_rerank/README.html)

Here are the steps:
* [Loading the reranking model](#loading-the-reranking-model)
* [Lading retrieval results](#loading-retrieval-results)
* [Calculating reranking score](#calculating-the-re-ranking-scores)
* [Generating a reply on the reranked documents](#using-merged-results-to-generate-a-reply)

## Visual Improvements

In [1]:
from rich.console import Console
from rich_theme_manager import Theme, ThemeManager
import pathlib

theme_dir = pathlib.Path("themes")
theme_manager = ThemeManager(theme_dir=theme_dir)
dark = theme_manager.get("dark")

# Create a console with the dark theme
console = Console(theme=dark)

In [2]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

## Loading the Reranking model

In [3]:
from sentence_transformers import CrossEncoder 
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
console.print(cross_encoder.model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermediate(
            (dense): Linear(in_features=384, out_features=1536, bias=True)
            (intermediate_act_fn): GELUActivation()
          )
          (output): BertOutput(
            (dense): Linear(in_features=1536, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
    )
    (pooler): BertPooler(
      (dense): Linear(in_features=384, out_features=384, bias=True)
      (activation): Tanh()
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Linear(in_features=384, out_features=1, bias=True)
)

## Loading retrieval results

We will load the retrieval results from the previous Hybrid-Search notebook, to avoid repetition. We can ignore the scores of the dense and sparse index, as we will calculate the ranking score based on the text of the document/chunk.

In [4]:
import json
hybrid_search_results = {}
with open('data/dense_results.json') as f:
    dense_results = json.load(f)
    for doc in dense_results:
        hybrid_search_results[doc['id']] = doc
with open('data/sparse_results.json') as f:
    sparse_results = json.load(f)
    for doc in sparse_results:
        hybrid_search_results[doc['id']] = doc
console.print(hybrid_search_results)

{
    15: {
        'id': 15,
        'text': "3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length Passkey 
Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K 16K 20K 24K 28K 0 
5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance of Mixtral. (Left) Mixtral has 100% 
retrieval accuracy of the Passkey task regardless of the location of the passkey and length of the input sequence. 
(Right) The perplexity of Mixtral on the proof-pile dataset decreases monotonically as the context length 
increases.\n\nThe chunk discusses the long-range performance of the Mixtral model, demonstrating its ability to 
retrieve a passkey regardless of its location in a long input sequence, and showing that the model's perplexity on 
the proof-pile dataset decreases as the context length increases.",
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    },
    4: {
        'id': 4,
        'text': "Instruct under the Apache 2.0 license1, free for academic and commercial usage, ensuring broad 
accessibility and potential for diverse applications. To enable the community to run Mixtral with a fully 
open-source stack, we submitted changes to the vLLM project, which integrates Megablocks CUDA kernels for efficient
inference. Skypilot also allows the deployment of vLLM endpoints on any instance in the cloud. # 2 Architectural 
details Mixtral is based on a transformer architecture [31] and uses the same modifications as described in [18], 
with the notable exceptions that Mix- tral supports a fully dense context length of 32k tokens, and the feed- 
forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture parameters are 
summarized in Table 1.\n\nThis chunk describes the architectural details of the Mixtral language model, including 
its use of a transformer architecture with a 32k token context length and mixture-of-expert layers. It also 
mentions the model's open-source licensing and deployment options.",
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    },
    2: {
        'id': 2,
        'text': 'expertsâ ) to process the token and combine their output additively. This technique increases the 
number of parameters of a model while controlling cost and latency, as the model only uses a fraction of the total 
set of parameters per token. Mixtral is pretrained with multilingual data using a context size of 32k tokens. It 
either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In particular, 
Mixture of Experts Layer i gating inputs af outputs router expert\n\nThis chunk describes the key architectural 
details of the Mixtral model, a sparse mixture-of-experts language model that outperforms larger models like Llama 
2 70B and GPT-3.5 on various benchmarks.',
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    },
    6: {
        'id': 6,
        'text': 'Table 1: Model architecture. # j nâ G(x)i Â· Ei(x). i=0 Here, G(x)i denotes the n-dimensional 
output of the gating network for the i-th expert, and Ei(x) is the output of the i-th expert network. If the gating
vector is sparse, we can avoid computing the outputs of experts whose gates are zero. There are multiple 
alternative ways of implementing G(x) [6, 15, 35], but a simple and performant one is implemented by taking the 
softmax over the Top-K logits of a linear layer [28].\n\nThe chunk describes the architectural details of the 
Mixtral model, specifically the Sparse Mixture of Experts (SMoE) layer that is used in the model.',
        'metadata': {'title': 'Mixtral of Experts', 'arxiv_id': '2401.04088', 'references': ['1905.07830']}
    },
    7: {
        'id': 7,
        'text': 'We use G(x) := Softmax(TopK(x Â· Wg)), where (TopK(â ))

In [5]:
# This is the query that we used for the retrieval of the above documents
query = "What is context size of Mixtral?"

## Calculating the re-ranking scores

We are using the `cross_encoder` to calculate the match score.

In [6]:
pairs = [[query, doc['text']] for doc in hybrid_search_results.values()] 
scores = cross_encoder.predict(pairs) 

console.print(scores)

[ 5.065694   3.3688319  7.1048393 -4.116105  -4.3754992 -5.261075
 -3.7225761  3.1854553  1.79667   -2.5144258  2.563871   1.6508542
  2.3361566 -3.0395913  3.0869908 -2.4781265]

## Selecting top 3 reranked documents

In [7]:
# Combine scores with corresponding document IDs
results_with_scores = [
    (doc_id, hybrid_search_results[doc_id]['text'], score)
    for doc_id, score in zip(hybrid_search_results.keys(), scores)
]

# Sort results by score in descending order and take the top 3
top_results = sorted(results_with_scores, key=lambda x: x[2], reverse=True)[:3]


In [8]:
import numpy as np
from rich.table import Table
table = Table(title="Top 3 Documents after Reranking", show_lines=True)

table.add_column("ID", justify="right", style="cyan", no_wrap=True)
table.add_column("Score", justify="right", style="green", no_wrap=True)
table.add_column("Document", style="#e87d3e")

# Add rows to the table with top 3 results
for doc_id, text, score in top_results:
    table.add_row(str(doc_id), f"{score:.4f}", text)

console.print(table)

                                          Top 3 Documents after Reranking                                          
┏━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID ┃  Score ┃ Document                                                                                          ┃
┡━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  2 │ 7.1048 │ expertsâ ) to process the token and combine their output additively. This technique increases the │
│    │        │ number of parameters of a model while controlling cost and latency, as the model only uses a      │
│    │        │ fraction of the total set of parameters per token. Mixtral is pretrained with multilingual data   │
│    │        │ using a context size of 32k tokens. It either matches or exceeds the performance of Llama 2 70B   │
│    │        │ and GPT-3.5, over several benchmarks. In particular, Mixture of Experts Layer i gating inputs af  │
│    │        │ outputs router expert                                                                             │
│    │        │                                                                                                   │
│    │        │ This chunk describes the key architectural details of the Mixtral model, a sparse                 │
│    │        │ mixture-of-experts language model that outperforms larger models like Llama 2 70B and GPT-3.5 on  │
│    │        │ various benchmarks.                                                                               │
├────┼────────┼───────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 15 │ 5.0657 │ 3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length Passkey │
│    │        │ Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K  │
│    │        │ 16K 20K 24K 28K 0 5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance  │
│    │        │ of Mixtral. (Left) Mixtral has 100% retrieval accuracy of the Passkey task regardless of the      │
│    │        │ location of the passkey and length of the input sequence. (Right) The perplexity of Mixtral on    │
│    │        │ the proof-pile dataset decreases monotonically as the context length increases.                   │
│    │        │                                                                                                   │
│    │        │ The chunk discusses the long-range performance of the Mixtral model, demonstrating its ability to │
│    │        │ retrieve a passkey regardless of its location in a long input sequence, and showing that the      │
│    │        │ model's perplexity on the proof-pile dataset decreases as the context length increases.           │
├────┼────────┼───────────────────────────────────────────────────────────────────────────────────────────────────┤
│  4 │ 3.3688 │ Instruct under the Apache 2.0 license1, free for academic and commercial usage, ensuring broad    │
│    │        │ accessibility and potential for diverse applications. To enable the community to run Mixtral with │
│    │        │ a fully open-source stack, we submitted changes to the vLLM project, which integrates Megablocks  │
│    │        │ CUDA kernels for efficient inference. Skypilot also allows the deployment of vLLM endpoints on    │
│    │        │ any instance in the cloud. # 2 Architectural details Mixtral is based on a transformer            │
│    │        │ architecture [31] and uses the same modifications as described in [18], with the notable          │
│    │        │ exceptions that Mix- tral supports a fully dense context length of 32k tokens, and the feed-      │
│    │        │ forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture     │
│    │        │ parameters are summarized in Table 1.   

## Using merged results to generate a reply

We can now take the improved merged results and call the LLM to generate the reply to the user's query.

In [9]:
# define a variable to hold the search results for the generation model
search_results = [doc[1] for doc in top_results]

In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
# Now time to connect to the large language model
from openai import OpenAI
from rich.text import Text

client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are chatbot, an research expert. Your top priority is to help guide users to understand reserach papers."},
        {"role": "user", "content": query},
        {"role": "assistant", "content": str(search_results)}
    ]
)

response_text = Text(completion.choices[0].message.content)

In [12]:
from rich.panel import Panel

panel = Panel(response_text, title=f"Hybrid Search with Reranking Reply to \"{query}\"")
console.print(panel)

╭─────────────────── Hybrid Search with Reranking Reply to "What is context size of Mixtral?" ────────────────────╮
│ The context size of Mixtral is 32,000 tokens. The model is pretrained with multilingual data using this context │
│ size, enabling it to achieve high performance on various benchmarks.                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯